In [12]:
# Importing important libraries

import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary
import nltk
from nltk.stem import PorterStemmer
import tqdm
import json
import string
from torch.utils.data import DataLoader, Dataset
import random

In [13]:
with open('university_support.json', 'r') as f:
    data = json.load(f)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon']},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
    'you are made by',
    'you are made by wh

## Setting up the tokenization and stem function

In [14]:
stemmer = PorterStemmer()

def tokenize(text: str):
    return nltk.tokenize.word_tokenize(text)

def stem(input: list):
    return stemmer.stem(input)

In [15]:
# Testing stem and tokenize function
[stem(i.lower()) for i in tokenize("They're a group of boys")]

['they', "'re", 'a', 'group', 'of', 'boy']

In [16]:
allwords = []
tags = []
xy = []

for intents in data['intents']:
    if intents['tag'] not in tags:
        tag = intents['tag']
        tags.append(tag)
    
    for patterns in intents['patterns']:
        word = tokenize(patterns.lower())
        word = [stem(x) for x in word if x not in list(string.punctuation)]
        allwords.extend(word)
        xy.append(([i for i in word], tag))

print(xy)
allwords = sorted(set(allwords))
allwords

[(['hi'], 'greeting'), (['how', 'are', 'you'], 'greeting'), (['is', 'anyon', 'there'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['what', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyi'], 'greeting'), (['whatsup'], 'greeting'), ([], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodby'], 'goodbye'), (['i', 'am', 'leav'], 'goodbye'), (['bye'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'develop'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creator'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'develop'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creator'], 'creator'), (['who', 'creat', 'you'], 'creator'), (['your', 'develop']

["'s",
 'a',
 'about',
 'ac',
 'activ',
 'address',
 'admis',
 'admiss',
 'against',
 'ai/ml',
 'allot',
 'am',
 'an',
 'and',
 'ani',
 'antirag',
 'anyon',
 'are',
 'ass',
 'asshol',
 'at',
 'attend',
 'automobil',
 'avail',
 'averag',
 'be',
 'between',
 'big',
 'bitch',
 'book',
 'boy',
 'branch',
 'bring',
 'build',
 'by',
 'bye',
 'cafetaria',
 'call',
 'campu',
 'can',
 'canteen',
 'capac',
 'case',
 'casual',
 'ce',
 'chat',
 'chemic',
 'civil',
 'code',
 'colleg',
 'come',
 'committ',
 'committe',
 'comp',
 'compani',
 'comput',
 'conduct',
 'contact',
 'cours',
 'creat',
 'creator',
 'cya',
 'date',
 'day',
 'design',
 'detail',
 'develop',
 'differ',
 'distanc',
 'do',
 'document',
 'doe',
 'done',
 'dress',
 'dresscod',
 'dumb',
 'dure',
 'each',
 'eat',
 'end',
 'engin',
 'event',
 'exam',
 'extc',
 'facil',
 'far',
 'fee',
 'first',
 'floor',
 'food',
 'for',
 'fourth',
 'from',
 'fuck',
 'fucker',
 'function',
 'game',
 'get',
 'girl',
 'give',
 'go',
 'good',
 'goodby',


In [17]:
def bag_of_words(allwords: list, input: list) -> torch.tensor:
    word = torch.zeros(size=[len(allwords)])
    for idx, text in enumerate(allwords):
        if text in input:
            word[idx] = 1
    return word

bag_of_words(allwords, ['is', 'anyon', 'there', 'how', 'are', 'you'])

x_train = []
y_train = []
for text, tag in xy:
    text = bag_of_words(allwords, text)
    x_train.append(text)
    y_train.append(tags.index(tag))

x_train[10], y_train[10]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.,

In [18]:
# Building the dataset and dataloader
torch.manual_seed(1)
torch.cuda.manual_seed(1)

class ChatDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)
    
    def __str__(self):
        return f'Data set of {self.__class__.__name__}'
    
dataset = ChatDataset(x=x_train, y=y_train)

chat_dataloader = DataLoader(dataset=dataset, shuffle=True, batch_size=8)
print(f'Length of dataset: {len(dataset)} | Length of dataloader: {len(chat_dataloader)}')
print(dataset[0])

Length of dataset: 405 | Length of dataloader: 51
(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [19]:
# Hyper parameters
input_features = len(allwords)
output_features = len(tags)
hidden_units = 256

# The model

class ChatbotV1(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=input_features, out_features=hidden_units)
        self.layer_2 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.layer_3 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.classifier = nn.Linear(in_features=hidden_units, out_features=output_features)


    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(F.relu(self.layer_3(F.relu(self.layer_2(F.relu(self.layer_1(x)))))))

model_v1 = ChatbotV1(input_features=input_features, output_features=output_features, hidden_units=hidden_units)
summary(model_v1, input_size=[len(allwords)])

Layer (type:depth-idx)                   Output Shape              Param #
ChatbotV1                                [38]                      --
├─Linear: 1-1                            [256]                     63,744
├─Linear: 1-2                            [256]                     65,792
├─Linear: 1-3                            [256]                     65,792
├─Linear: 1-4                            [38]                      9,766
Total params: 205,094
Trainable params: 205,094
Non-trainable params: 0
Total mult-adds (M): 50.38
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.82
Estimated Total Size (MB): 0.83

In [20]:
# The optimizer and loss function
criteron = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_v1.parameters(), lr=1e-3)

In [21]:
# Training loop

epochs = 50

for epoch in tqdm.tqdm(range(epochs)):
    model_v1.train()
    train_loss = 0
    train_acc = 0
    for x, y in chat_dataloader:
        y_logits = model_v1(x)
        y_preds = torch.argmax(torch.softmax(y_logits, dim=1), dim=-1)
        loss = criteron(y_logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # calculate loss and accuracy
        train_loss += loss.item()
        train_acc += torch.eq(y_preds, y).sum() / len(y)

    train_loss = train_loss / len(chat_dataloader)
    train_acc = train_acc / len(chat_dataloader)

    model_v1.eval()
    if (epoch + 1) % 10 == 0:
        print(f'EPOCH {epoch + 1} || Loss: {train_loss:.4f} | Accuracy: {train_acc * 100:.2f}%')


 20%|██        | 10/50 [00:05<00:21,  1.89it/s]

EPOCH 10 || Loss: 0.1153 | Accuracy: 97.06%


 40%|████      | 20/50 [00:10<00:17,  1.76it/s]

EPOCH 20 || Loss: 0.0190 | Accuracy: 99.26%


 60%|██████    | 30/50 [00:15<00:09,  2.11it/s]

EPOCH 30 || Loss: 0.0059 | Accuracy: 99.75%


 80%|████████  | 40/50 [00:20<00:05,  1.92it/s]

EPOCH 40 || Loss: 0.0058 | Accuracy: 99.75%


100%|██████████| 50/50 [00:26<00:00,  1.90it/s]

EPOCH 50 || Loss: 0.0061 | Accuracy: 99.51%


In [22]:
def chat(text):
    text = tokenize(text)
    text = [stem(w) for w in text]
    text = bag_of_words(allwords, text)

    model_v1.eval()
    with torch.inference_mode():
        logit = model_v1(text)

    pred = torch.argmax(torch.softmax(logit, dim=-1))
    c_tag = tags[pred]
    print(f'Index: {pred} | Class: {c_tag}')
    res = [random.choice(x["responses"]) for x in data["intents"] if x["tag"] == c_tag][0]
    print(f'Response: {res}')
    return res
    

chat('Who created you')

Index: 2 | Class: creator
Response: College students


'College students'

In [23]:
from pathlib import Path

MODEL_SAVE_PATH = Path('model')
MODEL_SAVE_PATH.mkdir(exist_ok=True, parents=True)
MODEL_NAME = 'ChatbotV1'

In [25]:
# Saving the model
torch.save(model_v1.state_dict(), MODEL_SAVE_PATH/MODEL_NAME)

In [26]:
loaded_model = ChatbotV1(input_features, output_features, hidden_units)
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH/MODEL_NAME))

def chat(text):
    text = tokenize(text)
    text = [stem(w) for w in text]
    text = bag_of_words(allwords, text)

    loaded_model.eval()
    with torch.inference_mode():
        logit = loaded_model(text)

    pred = torch.argmax(torch.softmax(logit, dim=-1))
    c_tag = tags[pred]
    print(f'Index: {pred} | Class: {c_tag}')
    res = [random.choice(x["responses"]) for x in data["intents"] if x["tag"] == c_tag][0]
    print(f'Response: {res}')
    return res
    

chat('Who created you')

Index: 2 | Class: creator
Response: College students


'College students'